<a href="https://colab.research.google.com/github/subin-saju/NLP/blob/main/NLP_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 24.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
pip install PyPDF2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.9 MB/s eta 0:00:00


In [3]:
import PyPDF2
file=open('/content/drive/MyDrive/US_Declaration.pdf','rb')
reader=PyPDF2.PdfReader(file)
text=[]
for i in range(len(reader.pages)):
    p1=reader.pages[i]
    text.append(p1.extract_text())


In [5]:
import spacy
nlp = spacy.load('en_core_web_sm')
doc= nlp(' '.join(text))
for token in doc:
    print(f'{token.text:{10}}, {token.pos_:{20}}, {spacy.explain(token.pos_)}')

Declaration, PROPN               , proper noun
of        , ADP                 , adposition
Independence, PROPN               , proper noun

         , SPACE               , space
IN        , ADP                 , adposition
CONGRESS  , PROPN               , proper noun
,         , PUNCT               , punctuation
July      , PROPN               , proper noun
4         , NUM                 , numeral
,         , PUNCT               , punctuation
1776      , NUM                 , numeral
.         , PUNCT               , punctuation
 
        , SPACE               , space
The       , DET                 , determiner
unanimous , ADJ                 , adjective
Declaration, PROPN               , proper noun
of        , ADP                 , adposition
the       , DET                 , determiner
thirteen  , NUM                 , numeral
united    , ADJ                 , adjective
States    , PROPN               , proper noun
of        , ADP                 , adposition
America   , PROPN 

In [6]:
pos_counts=doc.count_by(spacy.attrs.POS)
pos_counts
for k,v in pos_counts.items():
    print(f'{doc.vocab[k].text:{10}},{v:{20}}')

PROPN     ,                 299
ADP       ,                 218
SPACE     ,                 179
PUNCT     ,                 190
NUM       ,                  13
DET       ,                 130
ADJ       ,                 102
SCONJ     ,                  28
NOUN      ,                 266
PRON      ,                 139
VERB      ,                 161
PART      ,                  46
AUX       ,                  71
CCONJ     ,                  59
ADV       ,                  39
X         ,                   7


In [19]:
do=nlp(u"my organisation is thj")
for tokens in do.ents:
  print(tokens.text,tokens.label_)

In [16]:
from spacy.tokens import Span
ORG=do.vocab.strings['ORG']
ORG
new_ent=Span(do,2,3,label=ORG)
doc.ents=list(do.ents)+[new_ent]

In [39]:
doc=nlp(u"our company created  new brand k-i ki  ki")

if doc.ents:
  print('Entities',[(e.text,e.label_) for e in doc.ents])
else:
  print('No entities found')

No entities found


In [45]:
from re import I
from spacy.matcher import PhraseMatcher
matcher=PhraseMatcher(nlp.vocab)
p=['vaccum_cleaner','vaccum cleaner']
p=[nlp(text) for text in ['ki','k-i']] # Convert strings to Doc objects
matcher.add('newproduct',None,*p)
found_matches=matcher(doc)
found_matches
new_ents=[Span(doc,match[1],match[2],label='PRODUCT') for match in found_matches]
doc.ents= list(doc.ents)+ new_ents
if doc.ents:
  print('Entities',[(e.text,e.label_) for e in doc.ents])
else:
  print('No entities found')


Entities [('k-i', 'PRODUCT'), ('ki', 'PRODUCT'), ('k-i', 'PRODUCT')]


In [46]:
from spacy import displacy
displacy.render(doc,style='ent',jupyter=True)

In [59]:
doc=nlp(u'" manager id  doing the right things; leader  "')
for sent in doc.sents:
  print(sent)

" manager id  doing the right things;
leader  "


In [56]:
from spacy.language import Language

@Language.component("custom_boundaries")  # Register the function as a component
def set_custom_boundaries(doc):
  for token in doc[:-1]:
    if token.text == ';':
      doc[token.i+1].is_sent_start=True
  return doc
nlp.add_pipe("custom_boundaries", before='parser')  # Add the component by its name
nlp.pipe_names

['tok2vec',
 'tagger',
 'custom_boundaries',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ner']

In [60]:
for sent in doc.sents:
  print(sent)

" manager id  doing the right things;
leader  "
